In [3]:
# ==================================
# 1️⃣ SETUP: Install Python packages
# ==================================
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-distutils python3.10-dev -y
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10

# Upgrade pip
!python3.10 -m pip install --upgrade pip

# Install LangChain and dependencies explicitly
!python3.10 -m pip install langchain --upgrade

# Check versions
!python3.10 --version
!python3.10 -m pip show langchain

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [8]:
!pip install langchain_core
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [13]:
# ==============================================
# 2️⃣ IMPORTS
# ==============================================
import requests
from google.colab import userdata
from langchain.output_parsers import CommaSeparatedListOutputParser, DatetimeOutputParser

In [14]:
# ==============================================
# 3️⃣ API KEY
# ==============================================
api_key = userdata.get("OPENAI_API_KEY")
api_url = "https://api.aimlapi.com/v1/chat/completions"

In [15]:
# ==============================================
# 4️⃣ HELPER: Call AIMLAPI
# ==============================================
def call_aimlapi(prompt: str, model: str = "gpt-3.5-turbo") -> str:
    headers = {"Authorization": f"Bearer {api_key}"}
    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0
    }
    response = requests.post(api_url, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json()
    return data['choices'][0]['message']['content']

In [16]:
# ==============================================
# 5️⃣ STRING OUTPUT PARSER
# ==============================================
print("----- STRING OUTPUT PARSER -----")
string_prompt = "Write a short greeting message for students learning LangChain."
response = call_aimlapi(string_prompt)
print("LLM Response:", response)
parsed_string = response
print("Parsed Output:", parsed_string)

----- STRING OUTPUT PARSER -----
LLM Response: Hello LangChain students! Welcome to the exciting world of language learning. We are thrilled to have you on this journey with us. Remember, practice makes perfect, so keep pushing yourself and don't be afraid to make mistakes. Enjoy the process and have fun exploring new languages!
Parsed Output: Hello LangChain students! Welcome to the exciting world of language learning. We are thrilled to have you on this journey with us. Remember, practice makes perfect, so keep pushing yourself and don't be afraid to make mistakes. Enjoy the process and have fun exploring new languages!


In [17]:
# ==============================================
# 6️⃣ CSV / Comma-Separated List PARSER
# ==============================================
print("\n----- CSV OUTPUT PARSER -----")
csv_prompt = "Give me 5 programming languages separated by commas."
response = call_aimlapi(csv_prompt)
print("LLM Response:", response)
csv_parser = CommaSeparatedListOutputParser()
parsed_list = csv_parser.parse(response)
print("Parsed Output (List):", parsed_list)


----- CSV OUTPUT PARSER -----
LLM Response: Python, Java, C++, JavaScript, Ruby
Parsed Output (List): ['Python', 'Java', 'C++', 'JavaScript', 'Ruby']


In [19]:
# ==============================================
# 7️⃣ DATETIME OUTPUT PARSER
# ==============================================
print("\n----- DATETIME OUTPUT PARSER -----")
datetime_prompt = "Give today's date in full ISO 8601 format (YYYY-MM-DDTHH:MM:SS.sssZ)."
response = call_aimlapi(datetime_prompt)
print("LLM Response:", response)

from langchain.output_parsers import DatetimeOutputParser
datetime_parser = DatetimeOutputParser()
parsed_datetime = datetime_parser.parse(response)
print("Parsed Output:", parsed_datetime)


----- DATETIME OUTPUT PARSER -----
LLM Response: 2023-11-01T00:00:00.000Z
Parsed Output: 2023-11-01 00:00:00


In [21]:
import json
from datetime import datetime
from langchain.output_parsers import CommaSeparatedListOutputParser

print("\n----- COMBINED PARSERS WORKFLOW (Structured JSON) -----")

# Prompt LLM to return JSON
combined_prompt = """
Return the following information as a valid JSON object with real values:
{
    "fruits": ["fruit1", "fruit2", "fruit3"],
    "event_date": "YYYY-MM-DD",
    "message": "short welcome message"
}

Make sure to:
- Replace "YYYY-MM-DD" with today's date in actual format, e.g., "2025-10-13".
- Replace placeholder fruits and message with real examples.
- Keys must be exactly: fruits, event_date, message
"""

response = call_aimlapi(combined_prompt)
print("LLM Raw Response:\n", response)

# Parse JSON
try:
    data = json.loads(response)
except json.JSONDecodeError:
    # Some LLMs may include extra text, extract JSON block
    import re
    match = re.search(r"\{.*\}", response, re.DOTALL)
    if match:
        data = json.loads(match.group())
    else:
        raise ValueError("Could not parse JSON from LLM response")

# Use LangChain parser for fruits (comma-separated)
fruits = CommaSeparatedListOutputParser().parse(", ".join(data["fruits"]))

# Parse date manually (YYYY-MM-DD)
event_date = datetime.strptime(data["event_date"], "%Y-%m-%d")

# Message is just a string
message = data["message"]

print("Parsed Fruits:", fruits)
print("Parsed Event Date:", event_date)
print("Parsed Message:", message)


----- COMBINED PARSERS WORKFLOW (Structured JSON) -----
LLM Raw Response:
 {
    "fruits": ["apple", "banana", "orange"], 
    "event_date": "2022-12-25", 
    "message": "Welcome to our fruit festival!"
}
Parsed Fruits: ['apple', 'banana', 'orange']
Parsed Event Date: 2022-12-25 00:00:00
Parsed Message: Welcome to our fruit festival!
